# Домашнее задание 2
### Андреев Никита

In [16]:
import sys
import time
import matplotlib.pyplot as plt
import numpy as np
import scipy
import scipy.optimize as opt
import scipy.sparse
import sklearn.datasets
from sklearn.model_selection import train_test_split

#### Определим функцию $\sigma$:

## $\sigma_w(x)=\frac{e^{w^tx}}{1+e^{w^tx}}=\frac{1}{1+e^{-w^tx}}$

Тогда
$1-\sigma_w(x)=1-\frac{1}{1+e^{-w^tx}}=\frac{1}{1+e^{w^tx}}=\sigma_w(-x)$

#### Функция, которую необходимо минимизировать:
## $f(w)=-\frac{1}{N}\sum\limits_1^n\log(\sigma_w(y_i x_i))$, $y_i =\pm 1$

#### Посчитаем производные:

### $\nabla\sigma_w(x)=\frac{e^{-w^tx}}{(1+e^{-w^tx})^2}x = \sigma_w(x) \sigma_w(-x)x$

### $\nabla f(w) = -\frac{1}{N}\sum\limits_1^n\sigma_w(-y_i x_i)y_i x_i$

### $\nabla^2 f(w) = \frac{1}{N}\sum\limits_1^n\sigma_w(y_i x_i)\sigma_w(-y_i x_i)x_i x_i^t=
\frac{1}{N}\sum\limits_1^n\sigma_w(x_i)\sigma_w(-x_i)x_i x_i^t$

Матрица $X$ для датасета a1a разрежена, поэтому я попытался определить функции так, чтобы это учесть:

In [17]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def function(w, X, labels):
    Xw = X.dot(w)
    return -1 / X.shape[0] * np.sum([np.log(sigmoid(l * xw)) for xw, l in zip(Xw, labels)])


def gradient(w, X, labels):
    Xw = X.dot(w)
    coeffs = np.array([l * sigmoid(-l * xw) for xw, l in zip(Xw, labels)])
    X1 = X.multiply(coeffs.reshape(-1, 1))
    return -1 / X.shape[0] * np.array(X1.sum(axis=0)).reshape(X.shape[1])


def hessian(w, X, labels):
    Xw = X.dot(w)
    coeffs = np.array([sigmoid(xw) * sigmoid(-xw) for xw in Xw])
    return 1 / X.shape[0] * np.sum([np.outer(x, x) * coeffs[i] for i, x in enumerate(X.todense())], axis=0)
    #я не нашёл инструментов чтобы эффективно посчитать эту строчку эффективно для разреженных матриц
    #но в матрице X датасетa a1a есть полностью нулевые столбцы, поэтому гессиан будет всегда вырожден
    #и мы не сможем применить метод ньютона, и гессиан считать не придется


#### Тесты для проверки правильности вычисления производных:

In [37]:
#численное дифференцирование
def der(fun, point, epsilon=np.sqrt(sys.float_info.epsilon)):
    return (fun(point + epsilon) - fun(point)) / epsilon

#считает среднюю относительную погрешность по всем кординатным осям в рандомной точке из [-R, R]^dim
def check_gradient(fun, grad, R, dim, args=(), diff_eps=np.sqrt(sys.float_info.epsilon)):
    w = np.random.random(dim)
    w = (2 * w - 1) * R
    dw = np.eye(dim)
    g = grad(w, *args)
    norm = np.dot(g, g)
    difs = [np.abs((np.dot(g, dw_i) - der(lambda t: fun(w + t * dw_i, *args), 0, diff_eps))) for dw_i in dw]
    return np.average(difs) / norm

#считает среднюю относительную погрешность по всем парам кординатных осей в рандомной точке из [-R, R]^dim
def check_hessian(grad, hess, R, dim, args=(), diff_eps=np.sqrt(sys.float_info.epsilon)):
    w = np.random.random(dim)
    w = (2 * w - 1) * R
    dw = np.eye(dim)
    h = hess(w, *args)
    norm = np.dot(h.flatten(), h.flatten())

    def xAy(A, x, y):
        return np.dot(x, np.dot(A, y))

    difs = [np.abs((xAy(h, dw1, dw2) - der(lambda t: np.dot(grad(w + t * dw1, *args), dw2), 0, diff_eps))) for dw1 in dw
            for dw2 in dw]
    return np.average(difs) / norm


### В качестве данных впредь будем использовать три датасета:

a1a:

In [19]:
a1a = sklearn.datasets.load_svmlight_file('../data/a1a.txt')
X = a1a[0]
dummy = scipy.sparse.coo_matrix([[1] for i in range(X.shape[0])])
X_a1a = scipy.sparse.hstack([X, dummy])
labels_a1a = a1a[1]

breast cancer:
(координаты нормироаны от -1 до 1)

In [35]:
breast_cancer = sklearn.datasets.load_svmlight_file('../data/breast-cancer_scale.txt')
X = breast_cancer[0]
dummy = scipy.sparse.csr_matrix([[1] for i in range(X.shape[0])])
X_cancer = scipy.sparse.hstack([X, dummy])
labels_cancer = breast_cancer[1]-3

Случайный сгенерированный:

In [21]:
def random_dataset(alpha, beta):
    xs = np.random.normal(size=(1000, alpha.shape[0]))
    labels = np.array([np.sign(np.dot(alpha, x) + beta) for x in xs])
    return xs, labels


In [22]:
alpha = 2 * np.random.random(10) - 1
beta = 2 * np.random.random() - 1
X, labels_rand = random_dataset(alpha, beta)
dummy = scipy.sparse.csr_matrix([[1] for i in range(X.shape[0])])
X_rand = scipy.sparse.hstack([X, dummy])

Посмотрим, правильно ли посчитаны градиент и гессиан:

In [33]:
print('погрешность градиента =', check_gradient(function, gradient, 1, X_a1a.shape[1], args=[X_a1a, labels_a1a]))
print('погрешность гессиана =', check_hessian(gradient, hessian, 1, X_a1a.shape[1], args=[X_a1a, labels_a1a]))
#долго - большая размерность

погрешность градиента = 4.4215166133432194e-09
погрешность гессиана = 1.2081936227510622e-09


In [36]:
print('погрешность градиента =', check_gradient(function, gradient, 1, X_cancer.shape[1], args=[X_cancer, labels_cancer]))
print('погрешность гессиана =', check_hessian(gradient, hessian, 1, X_cancer.shape[1], args=[X_cancer, labels_cancer]))

погрешность градиента = 7.711015682828957e-09
погрешность гессиана = 3.357645728961336e-08


In [34]:
print('погрешность градиента =', check_gradient(function, gradient, 1, X_rand.shape[1], args=[X_rand, labels_rand]))
print('погрешность гессиана =', check_hessian(gradient, hessian, 1, X_rand.shape[1], args=[X_rand, labels_rand]))

погрешность градиента = 2.5157666787973703e-08
погрешность гессиана = 3.7431631029304215e-08


Результаты говорят о том, что всё правильно